In [ ]:
# run first 5 cells 
# add symbol variables here

# ============================================================================
# CELL 1: Configuration & Imports
# ============================================================================

import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import json
import os
import warnings
from pathlib import Path

warnings.filterwarnings('ignore')

# Configuration
GEMINI_API_KEY = None  # Set to your API key or leave None for rule-based ranking
DATA_DIR = 'data'
USE_AI_RANKING = False  # Set True if you have Gemini API key

# Optional: Import Gemini if API key provided
if GEMINI_API_KEY:
    try:
        from google import genai
        genai_client = genai.Client(api_key=GEMINI_API_KEY)
        USE_AI_RANKING = True
        print("✅ Gemini AI loaded")
    except ImportError:
        print("⚠️  google-generativeai not installed. Using rule-based ranking.")
        USE_AI_RANKING = False

# Create data directories
Path(f"{DATA_DIR}/signals").mkdir(parents=True, exist_ok=True)
Path(f"{DATA_DIR}/indicators").mkdir(parents=True, exist_ok=True)
Path(f"{DATA_DIR}/exports").mkdir(parents=True, exist_ok=True)

print("✅ Configuration loaded")


# ============================================================================
# CELL 2: Fetch Data & Calculate Indicators
# ============================================================================

def fetch_data(symbol, period='1mo'):
    """Fetch stock data from yfinance"""
    ticker = yf.Ticker(symbol)
    df = ticker.history(period=period)
    
    if df.empty:
        raise ValueError(f"No data found for {symbol}")
    
    return df


def calculate_indicators(df):
    """Calculate all technical indicators"""
    
    # Moving Averages
    for period in [5, 10, 20, 50, 100, 200]:
        df[f'SMA_{period}'] = df['Close'].rolling(window=period).mean()
        df[f'EMA_{period}'] = df['Close'].ewm(span=period, adjust=False).mean()
    
    # RSI
    delta = df['Close'].diff()
    gain = delta.where(delta > 0, 0).rolling(window=14).mean()
    loss = -delta.where(delta < 0, 0).rolling(window=14).mean()
    rs = gain / loss
    df['RSI'] = 100 - (100 / (1 + rs))
    
    # MACD
    exp1 = df['Close'].ewm(span=12, adjust=False).mean()
    exp2 = df['Close'].ewm(span=26, adjust=False).mean()
    df['MACD'] = exp1 - exp2
    df['MACD_Signal'] = df['MACD'].ewm(span=9, adjust=False).mean()
    df['MACD_Hist'] = df['MACD'] - df['MACD_Signal']
    
    # Bollinger Bands
    df['BB_Middle'] = df['Close'].rolling(window=20).mean()
    bb_std = df['Close'].rolling(window=20).std()
    df['BB_Upper'] = df['BB_Middle'] + (bb_std * 2)
    df['BB_Lower'] = df['BB_Middle'] - (bb_std * 2)
    df['BB_Width'] = df['BB_Upper'] - df['BB_Lower']
    
    # Stochastic
    low_14 = df['Low'].rolling(window=14).min()
    high_14 = df['High'].rolling(window=14).max()
    df['Stoch_K'] = 100 * ((df['Close'] - low_14) / (high_14 - low_14))
    df['Stoch_D'] = df['Stoch_K'].rolling(window=3).mean()
    
    # ADX
    high_low = df['High'] - df['Low']
    high_close = np.abs(df['High'] - df['Close'].shift())
    low_close = np.abs(df['Low'] - df['Close'].shift())
    ranges = pd.concat([high_low, high_close, low_close], axis=1)
    true_range = np.max(ranges, axis=1)
    
    plus_dm = df['High'].diff()
    minus_dm = -df['Low'].diff()
    plus_dm[plus_dm < 0] = 0
    minus_dm[minus_dm < 0] = 0
    
    tr14 = true_range.rolling(14).sum()
    plus_di = 100 * (plus_dm.rolling(14).sum() / tr14)
    minus_di = 100 * (minus_dm.rolling(14).sum() / tr14)
    dx = 100 * np.abs(plus_di - minus_di) / (plus_di + minus_di)
    df['ADX'] = dx.rolling(14).mean()
    df['Plus_DI'] = plus_di
    df['Minus_DI'] = minus_di
    
    # ATR
    df['ATR'] = true_range.rolling(14).mean()
    
    # Volume
    df['Volume_MA_20'] = df['Volume'].rolling(window=20).mean()
    df['Volume_MA_50'] = df['Volume'].rolling(window=50).mean()
    
    # OBV
    df['OBV'] = (np.sign(df['Close'].diff()) * df['Volume']).fillna(0).cumsum()
    
    # VWAP
    df['VWAP'] = (df['Volume'] * (df['High'] + df['Low'] + df['Close']) / 3).cumsum() / df['Volume'].cumsum()
    
    # Price Changes
    df['Price_Change'] = df['Close'].pct_change() * 100
    df['Price_Change_5d'] = ((df['Close'] - df['Close'].shift(5)) / df['Close'].shift(5)) * 100
    
    # Volatility
    df['Volatility'] = df['Close'].pct_change().rolling(20).std() * np.sqrt(252) * 100
    
    # Distance from MAs
    for period in [10, 20, 50, 200]:
        df[f'Dist_SMA_{period}'] = ((df['Close'] - df[f'SMA_{period}']) / df[f'SMA_{period}']) * 100
    
    # 52-week highs/lows
    df['High_52w'] = df['High'].rolling(window=252).max()
    df['Low_52w'] = df['Low'].rolling(window=252).min()
    
    return df

print("✅ Data & indicator functions loaded")


# ============================================================================
# CELL 3: Detect Trading Signals
# ============================================================================

def detect_signals(df):
    """Detect all trading signals"""
    current = df.iloc[-1]
    prev = df.iloc[-2]
    prev2 = df.iloc[-3] if len(df) > 2 else prev
    
    signals = []
    
    # Moving Average Crossovers
    if len(df) > 200:
        if prev['SMA_50'] <= prev['SMA_200'] and current['SMA_50'] > current['SMA_200']:
            signals.append({
                'signal': 'GOLDEN CROSS',
                'desc': '50 MA crossed above 200 MA',
                'strength': 'STRONG BULLISH',
                'category': 'MA_CROSS'
            })
        
        if prev['SMA_50'] >= prev['SMA_200'] and current['SMA_50'] < current['SMA_200']:
            signals.append({
                'signal': 'DEATH CROSS',
                'desc': '50 MA crossed below 200 MA',
                'strength': 'STRONG BEARISH',
                'category': 'MA_CROSS'
            })
    
    if prev['Close'] <= prev['SMA_20'] and current['Close'] > current['SMA_20']:
        signals.append({
            'signal': 'PRICE ABOVE 20 MA',
            'desc': 'Price crossed above 20-day MA',
            'strength': 'BULLISH',
            'category': 'MA_CROSS'
        })
    
    if prev['Close'] >= prev['SMA_20'] and current['Close'] < current['SMA_20']:
        signals.append({
            'signal': 'PRICE BELOW 20 MA',
            'desc': 'Price crossed below 20-day MA',
            'strength': 'BEARISH',
            'category': 'MA_CROSS'
        })
    
    # RSI Signals
    if current['RSI'] < 30:
        signals.append({
            'signal': 'RSI OVERSOLD',
            'desc': f"RSI at {current['RSI']:.1f}",
            'strength': 'BULLISH',
            'category': 'RSI'
        })
    
    if current['RSI'] > 70:
        signals.append({
            'signal': 'RSI OVERBOUGHT',
            'desc': f"RSI at {current['RSI']:.1f}",
            'strength': 'BEARISH',
            'category': 'RSI'
        })
    
    if current['RSI'] < 20:
        signals.append({
            'signal': 'RSI EXTREME OVERSOLD',
            'desc': f"RSI at {current['RSI']:.1f}",
            'strength': 'STRONG BULLISH',
            'category': 'RSI'
        })
    
    if current['RSI'] > 80:
        signals.append({
            'signal': 'RSI EXTREME OVERBOUGHT',
            'desc': f"RSI at {current['RSI']:.1f}",
            'strength': 'STRONG BEARISH',
            'category': 'RSI'
        })
    
    # MACD Signals
    if prev['MACD'] <= prev['MACD_Signal'] and current['MACD'] > current['MACD_Signal']:
        signals.append({
            'signal': 'MACD BULL CROSS',
            'desc': 'MACD crossed above signal',
            'strength': 'BULLISH',
            'category': 'MACD'
        })
    
    if prev['MACD'] >= prev['MACD_Signal'] and current['MACD'] < current['MACD_Signal']:
        signals.append({
            'signal': 'MACD BEAR CROSS',
            'desc': 'MACD crossed below signal',
            'strength': 'BEARISH',
            'category': 'MACD'
        })
    
    if prev['MACD'] <= 0 and current['MACD'] > 0:
        signals.append({
            'signal': 'MACD ABOVE ZERO',
            'desc': 'MACD crossed into positive territory',
            'strength': 'BULLISH',
            'category': 'MACD'
        })
    
    if prev['MACD'] >= 0 and current['MACD'] < 0:
        signals.append({
            'signal': 'MACD BELOW ZERO',
            'desc': 'MACD crossed into negative territory',
            'strength': 'BEARISH',
            'category': 'MACD'
        })
    
    # Bollinger Bands
    if current['Close'] <= current['BB_Lower'] * 1.01:
        signals.append({
            'signal': 'AT LOWER BB',
            'desc': f"Price at ${current['BB_Lower']:.2f}",
            'strength': 'BULLISH',
            'category': 'BOLLINGER'
        })
    
    if current['Close'] >= current['BB_Upper'] * 0.99:
        signals.append({
            'signal': 'AT UPPER BB',
            'desc': f"Price at ${current['BB_Upper']:.2f}",
            'strength': 'BEARISH',
            'category': 'BOLLINGER'
        })
    
    bb_width_avg = df['BB_Width'].tail(50).mean()
    if current['BB_Width'] < bb_width_avg * 0.7:
        signals.append({
            'signal': 'BB SQUEEZE',
            'desc': 'Bands narrowing - breakout pending',
            'strength': 'NEUTRAL',
            'category': 'BB_SQUEEZE'
        })
    
    # Volume Signals
    if current['Volume'] > current['Volume_MA_20'] * 2:
        signals.append({
            'signal': 'VOLUME SPIKE 2X',
            'desc': f"Vol: {current['Volume']:,.0f}",
            'strength': 'SIGNIFICANT',
            'category': 'VOLUME'
        })
    
    if current['Volume'] > current['Volume_MA_20'] * 3:
        signals.append({
            'signal': 'EXTREME VOLUME 3X',
            'desc': f"Vol: {current['Volume']:,.0f}",
            'strength': 'VERY SIGNIFICANT',
            'category': 'VOLUME'
        })
    
    if current['Price_Change'] > 2 and current['Volume'] > current['Volume_MA_20'] * 1.5:
        signals.append({
            'signal': 'VOLUME BREAKOUT',
            'desc': 'High volume + price up',
            'strength': 'STRONG BULLISH',
            'category': 'VOLUME'
        })
    
    if current['Price_Change'] < -2 and current['Volume'] > current['Volume_MA_20'] * 1.5:
        signals.append({
            'signal': 'VOLUME SELLOFF',
            'desc': 'High volume + price down',
            'strength': 'STRONG BEARISH',
            'category': 'VOLUME'
        })
    
    # Price Action
    if current['Price_Change'] > 5:
        signals.append({
            'signal': 'LARGE GAIN',
            'desc': f"+{current['Price_Change']:.1f}% today",
            'strength': 'STRONG BULLISH',
            'category': 'PRICE_ACTION'
        })
    
    if current['Price_Change'] < -5:
        signals.append({
            'signal': 'LARGE LOSS',
            'desc': f"{current['Price_Change']:.1f}% today",
            'strength': 'STRONG BEARISH',
            'category': 'PRICE_ACTION'
        })
    
    if current['Close'] >= current['High_52w'] * 0.999:
        signals.append({
            'signal': '52-WEEK HIGH',
            'desc': f"At ${current['Close']:.2f}",
            'strength': 'STRONG BULLISH',
            'category': 'RANGE'
        })
    
    if current['Close'] <= current['Low_52w'] * 1.001:
        signals.append({
            'signal': '52-WEEK LOW',
            'desc': f"At ${current['Close']:.2f}",
            'strength': 'STRONG BEARISH',
            'category': 'RANGE'
        })
    
    # Trend Strength
    if current['ADX'] > 25:
        trend = 'UP' if current['Close'] > current['SMA_50'] else 'DOWN'
        signals.append({
            'signal': f'STRONG {trend}TREND',
            'desc': f"ADX: {current['ADX']:.1f}",
            'strength': 'TRENDING',
            'category': 'TREND'
        })
    
    if current['ADX'] > 40:
        signals.append({
            'signal': 'VERY STRONG TREND',
            'desc': f"ADX: {current['ADX']:.1f}",
            'strength': 'EXTREME',
            'category': 'TREND'
        })
    
    # MA Alignment
    if current['SMA_10'] > current['SMA_20'] > current['SMA_50']:
        signals.append({
            'signal': 'MA ALIGNMENT BULLISH',
            'desc': '10 > 20 > 50 SMA',
            'strength': 'STRONG BULLISH',
            'category': 'MA_TREND'
        })
    
    if current['SMA_10'] < current['SMA_20'] < current['SMA_50']:
        signals.append({
            'signal': 'MA ALIGNMENT BEARISH',
            'desc': '10 < 20 < 50 SMA',
            'strength': 'STRONG BEARISH',
            'category': 'MA_TREND'
        })
    
    if len(df) > 200:
        if current['Close'] > current['SMA_200']:
            signals.append({
                'signal': 'ABOVE 200 SMA',
                'desc': 'Long-term uptrend',
                'strength': 'BULLISH',
                'category': 'MA_TREND'
            })
        
        if current['Close'] < current['SMA_200']:
            signals.append({
                'signal': 'BELOW 200 SMA',
                'desc': 'Long-term downtrend',
                'strength': 'BEARISH',
                'category': 'MA_TREND'
            })
    
    # Stochastic
    if current['Stoch_K'] < 20:
        signals.append({
            'signal': 'STOCHASTIC OVERSOLD',
            'desc': f"K at {current['Stoch_K']:.1f}",
            'strength': 'BULLISH',
            'category': 'STOCHASTIC'
        })
    
    if current['Stoch_K'] > 80:
        signals.append({
            'signal': 'STOCHASTIC OVERBOUGHT',
            'desc': f"K at {current['Stoch_K']:.1f}",
            'strength': 'BEARISH',
            'category': 'STOCHASTIC'
        })
    
    if prev['Stoch_K'] <= prev['Stoch_D'] and current['Stoch_K'] > current['Stoch_D']:
        signals.append({
            'signal': 'STOCHASTIC BULL CROSS',
            'desc': 'K crossed above D',
            'strength': 'BULLISH',
            'category': 'STOCHASTIC'
        })
    
    if prev['Stoch_K'] >= prev['Stoch_D'] and current['Stoch_K'] < current['Stoch_D']:
        signals.append({
            'signal': 'STOCHASTIC BEAR CROSS',
            'desc': 'K crossed below D',
            'strength': 'BEARISH',
            'category': 'STOCHASTIC'
        })
    
    # Distance from Moving Averages
    if current.get('Dist_SMA_20', 0) > 10:
        signals.append({
            'signal': 'OVEREXTENDED ABOVE 20MA',
            'desc': f"{current.get('Dist_SMA_20', 0):.1f}% above 20 MA",
            'strength': 'BEARISH',
            'category': 'MA_DISTANCE'
        })
    
    if current.get('Dist_SMA_20', 0) < -10:
        signals.append({
            'signal': 'OVEREXTENDED BELOW 20MA',
            'desc': f"{abs(current.get('Dist_SMA_20', 0)):.1f}% below 20 MA",
            'strength': 'BULLISH',
            'category': 'MA_DISTANCE'
        })
    
    # Gap Detection
    gap_up = (current['Open'] - prev['Close']) / prev['Close'] * 100
    if gap_up > 2:
        signals.append({
            'signal': 'GAP UP',
            'desc': f"Opened {gap_up:.1f}% higher",
            'strength': 'BULLISH',
            'category': 'GAP'
        })
    elif gap_up < -2:
        signals.append({
            'signal': 'GAP DOWN',
            'desc': f"Opened {abs(gap_up):.1f}% lower",
            'strength': 'BEARISH',
            'category': 'GAP'
        })
    
    return signals

print("✅ Signal detection function loaded")


# ============================================================================
# CELL 4: AI Signal Ranking (Optional)
# ============================================================================

def rank_signals_with_ai(symbol, signals, market_data):
    """Rank signals using Gemini AI (1-100 score)"""
    
    if not USE_AI_RANKING or not GEMINI_API_KEY:
        return rank_signals_local(signals)
    
    try:
        prompt = f"""Expert technical analyst scoring signals for {symbol}.

Market Data:
- Price: ${market_data['price']:.2f} | Change: {market_data['change']:.2f}%
- RSI: {market_data['rsi']:.1f} | MACD: {market_data['macd']:.4f} | ADX: {market_data['adx']:.1f}

Signals:
"""
        for i, sig in enumerate(signals, 1):
            prompt += f"{i}. {sig['signal']}: {sig['desc']} [{sig['strength']}]\n"
        
        prompt += """
Score each 1-100 based on actionability, reliability, timing, risk/reward.

Return ONLY JSON:
{"scores": [{"signal_number": 1, "score": 85, "reasoning": "Brief reason"}]}

Keep reasoning under 60 chars. Score ALL signals."""

        response = genai_client.models.generate_content(
            model='gemini-2.0-flash-exp',
            contents=prompt
        )
        
        response_text = response.text.strip()
        
        # Clean response
        if '```json' in response_text:
            response_text = response_text.split('```json')[1].split('```')[0].strip()
        elif '```' in response_text:
            response_text = response_text.split('```')[1].split('```')[0].strip()
        
        start_idx = response_text.find('{')
        end_idx = response_text.rfind('}')
        if start_idx != -1 and end_idx != -1:
            response_text = response_text[start_idx:end_idx+1]
        
        scores_data = json.loads(response_text)
        
        # Apply scores
        for score_item in scores_data.get('scores', []):
            sig_num = score_item['signal_number'] - 1
            if 0 <= sig_num < len(signals):
                signals[sig_num]['ai_score'] = score_item.get('score', 50)
                signals[sig_num]['ai_reasoning'] = score_item.get('reasoning', 'No reasoning')
        
        # Fill missing scores
        for signal in signals:
            if 'ai_score' not in signal:
                signal['ai_score'] = 50
                signal['ai_reasoning'] = 'Score not provided'
        
        # Sort by score
        signals.sort(key=lambda x: x['ai_score'], reverse=True)
        
        # Add ranks
        for rank, signal in enumerate(signals, 1):
            signal['rank'] = rank
        
        return signals
        
    except Exception as e:
        print(f"⚠️  AI ranking failed: {e}. Using rule-based.")
        return rank_signals_local(signals)


def rank_signals_local(signals):
    """Rule-based ranking fallback"""
    
    for signal in signals:
        score = 50
        
        strength = signal.get('strength', '')
        if 'EXTREME' in strength:
            score = 85
        elif 'STRONG' in strength:
            score = 75
        elif 'SIGNIFICANT' in strength or 'VERY' in strength:
            score = 65
        elif 'BULLISH' in strength or 'BEARISH' in strength:
            score = 55
        
        category = signal.get('category', '')
        if category in ['MA_CROSS', 'MACD', 'VOLUME']:
            score += 10
        elif category in ['TREND', 'MA_TREND']:
            score += 5
        
        signal['ai_score'] = min(score, 95)
        signal['ai_reasoning'] = 'Rule-based score'
    
    signals.sort(key=lambda x: x['ai_score'], reverse=True)
    
    for rank, signal in enumerate(signals, 1):
        signal['rank'] = rank
    
    return signals

print("✅ Signal ranking functions loaded")


# ============================================================================
# CELL 5: Main Analysis Function
# ============================================================================

def analyze_stock(symbol, period='1mo'):
    """Complete analysis pipeline"""
    
    print(f"\n{'='*60}")
    print(f"📊 Analyzing {symbol}...")
    print('='*60)
    
    # Fetch data
    df = fetch_data(symbol, period)
    print(f"✅ Fetched {len(df)} days of data")
    
    # Calculate indicators
    df = calculate_indicators(df)
    print("✅ Indicators calculated")
    
    # Detect signals
    signals = detect_signals(df)
    print(f"✅ Detected {len(signals)} signals")
    
    # Get current data
    current = df.iloc[-1]
    
    # Prepare market data for AI
    market_data = {
        'price': float(current['Close']),
        'change': float(current['Price_Change']),
        'rsi': float(current['RSI']),
        'macd': float(current['MACD']),
        'adx': float(current['ADX'])
    }
    
    # Rank signals
    if USE_AI_RANKING:
        print("🤖 AI ranking signals...")
        signals = rank_signals_with_ai(symbol, signals, market_data)
        print(f"✅ AI ranked {len(signals)} signals")
    else:
        signals = rank_signals_local(signals)
        print(f"✅ Rule-based ranked {len(signals)} signals")
    
    # Build result
    result = {
        'symbol': symbol,
        'timestamp': datetime.now().isoformat(),
        'price': float(current['Close']),
        'change': float(current['Price_Change']),
        'signals': signals,
        'summary': {
            'total_signals': len(signals),
            'bullish': sum(1 for s in signals if 'BULLISH' in s['strength']),
            'bearish': sum(1 for s in signals if 'BEARISH' in s['strength']),
            'avg_score': sum(s['ai_score'] for s in signals) / len(signals) if signals else 0
        },
        'indicators': {
            'rsi': float(current['RSI']),
            'macd': float(current['MACD']),
            'adx': float(current['ADX']),
            'volume': int(current['Volume']),
            'bb_upper': float(current['BB_Upper']),
            'bb_lower': float(current['BB_Lower']),
            'stoch_k': float(current['Stoch_K'])
        },
        'data': df
    }
    
    return result

print("✅ Main analysis function loaded")
print("\n🎉 All functions ready! Use: result = analyze_stock('AAPL', period='1mo')")

✅ Configuration loaded
✅ Data & indicator functions loaded
✅ Signal detection function loaded
✅ Signal ranking functions loaded
✅ Main analysis function loaded

🎉 All functions ready! Use: result = analyze_stock('AAPL', period='1mo')


In [ ]:
# ============================================================================
# CELL 6: Save & Export Functions
# ============================================================================

def save_results(result):
    """Save analysis results to local files"""
    symbol = result['symbol']
    date_str = datetime.now().strftime('%Y-%m-%d')
    timestamp_str = datetime.now().strftime('%H%M%S')
    
    signals_dir = f"{DATA_DIR}/signals/{date_str}"
    indicators_dir = f"{DATA_DIR}/indicators/{date_str}"
    Path(signals_dir).mkdir(parents=True, exist_ok=True)
    Path(indicators_dir).mkdir(parents=True, exist_ok=True)
    
    signals_file = f"{signals_dir}/{symbol}-signals-{timestamp_str}.json"
    save_data = {
        'symbol': result['symbol'],
        'timestamp': result['timestamp'],
        'price': result['price'],
        'change': result['change'],
        'signals': result['signals'],
        'summary': result['summary'],
        'indicators': result['indicators']
    }
    with open(signals_file, 'w') as f:
        json.dump(save_data, f, indent=2)
    
    indicators_file = f"{indicators_dir}/{symbol}-indicators-{timestamp_str}.csv"
    result['data'].to_csv(indicators_file)
    
    report_file = f"{signals_dir}/{symbol}-report-{timestamp_str}.txt"
    with open(report_file, 'w') as f:
        f.write(generate_text_report(result))
    
    return {
        'signals_json': signals_file,
        'indicators_csv': indicators_file,
        'report_txt': report_file
    }


def export_to_excel(result, filename=None):
    """Export results to formatted Excel workbook"""
    import openpyxl
    
    if filename is None:
        filename = f"{result['symbol']}_analysis_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
    
    filepath = f"{DATA_DIR}/exports/{filename}.xlsx"
    
    with pd.ExcelWriter(filepath, engine='openpyxl') as writer:
        summary_data = {
            'Metric': ['Symbol', 'Price', 'Change %', 'Date', 'Total Signals', 'Bullish', 'Bearish', 'Avg Score', 
                      'RSI', 'MACD', 'ADX', 'Volume'],
            'Value': [
                result['symbol'],
                f"${result['price']:.2f}",
                f"{result['change']:.2f}%",
                result['timestamp'][:10],
                result['summary']['total_signals'],
                result['summary']['bullish'],
                result['summary']['bearish'],
                f"{result['summary']['avg_score']:.1f}",
                f"{result['indicators']['rsi']:.1f}",
                f"{result['indicators']['macd']:.4f}",
                f"{result['indicators']['adx']:.1f}",
                result['indicators']['volume']
            ]
        }
        pd.DataFrame(summary_data).to_excel(writer, sheet_name='Summary', index=False)
        
        signals_df = pd.DataFrame([{
            'Rank': s['rank'],
            'Score': s['ai_score'],
            'Signal': s['signal'],
            'Description': s['desc'],
            'Strength': s['strength'],
            'Category': s['category'],
            'AI Reasoning': s.get('ai_reasoning', '')
        } for s in result['signals']])
        signals_df.to_excel(writer, sheet_name='Signals', index=False)
        
        indicators_df = result['data'][['Close', 'Volume', 'RSI', 'MACD', 'MACD_Signal', 
                                        'BB_Upper', 'BB_Lower', 'ADX', 'Stoch_K']].tail(30)
        indicators_df.to_excel(writer, sheet_name='Indicators')
    
    print(f"✅ Exported to {filepath}")
    return filepath


def export_to_csv(result, filename=None):
    """Export signals to CSV"""
    if filename is None:
        filename = f"{result['symbol']}_signals_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
    
    filepath = f"{DATA_DIR}/exports/{filename}.csv"
    
    signals_df = pd.DataFrame([{
        'Rank': s['rank'],
        'Score': s['ai_score'],
        'Signal': s['signal'],
        'Description': s['desc'],
        'Strength': s['strength'],
        'Category': s['category'],
        'AI_Reasoning': s.get('ai_reasoning', '')
    } for s in result['signals']])
    
    signals_df.to_csv(filepath, index=False)
    print(f"✅ Exported to {filepath}")
    return filepath


def generate_text_report(result):
    """Generate formatted text report"""
    report = f"""
{'='*80}
TECHNICAL ANALYSIS REPORT - {result['symbol']}
{'='*80}

Price: ${result['price']:.2f} ({result['change']:+.2f}%)
Date: {result['timestamp'][:19]}

{'='*80}
SIGNAL SUMMARY
{'='*80}

Total Signals: {result['summary']['total_signals']}
Bullish: {result['summary']['bullish']} | Bearish: {result['summary']['bearish']}
Average AI Score: {result['summary']['avg_score']:.1f}/100

{'='*80}
KEY INDICATORS
{'='*80}

RSI: {result['indicators']['rsi']:.1f}
MACD: {result['indicators']['macd']:.4f}
ADX: {result['indicators']['adx']:.1f}
Volume: {result['indicators']['volume']:,}

{'='*80}
TOP 20 SIGNALS (Ranked by AI)
{'='*80}

"""
    
    for i, sig in enumerate(result['signals'][:20], 1):
        score = sig['ai_score']
        indicator = "🔥" if score >= 80 else "⚡" if score >= 60 else "📊"
        
        report += f"\n#{i} {indicator} [Score: {score}/100]\n"
        report += f"Signal: {sig['signal']}\n"
        report += f"Description: {sig['desc']}\n"
        report += f"Category: {sig['category']} | Strength: {sig['strength']}\n"
        if sig.get('ai_reasoning'):
            report += f"AI Analysis: {sig['ai_reasoning']}\n"
        report += "-" * 80 + "\n"
    
    return report


def analyze_and_save(symbol, period='1mo'):
    """Complete analysis with automatic saving"""
    result = analyze_stock(symbol, period)
    files = save_results(result)
    
    print(f"\n{'='*60}")
    print("📊 ANALYSIS COMPLETE")
    print('='*60)
    print(f"\n💾 Files saved:")
    for key, path in files.items():
        print(f"  • {key}: {path}")
    
    display_summary(result)
    
    return result


def display_summary(result):
    """Display formatted summary"""
    print(f"\n{'='*60}")
    print(f"📊 {result['symbol']} Technical Analysis")
    print('='*60)
    
    change_icon = '🟢' if result['change'] > 0 else '🔴'
    print(f"{change_icon} Price: ${result['price']:.2f} ({result['change']:+.2f}%)")
    
    print(f"\n📈 Signal Summary:")
    print(f"• Total: {result['summary']['total_signals']}")
    print(f"• Bullish: {result['summary']['bullish']} | Bearish: {result['summary']['bearish']}")
    print(f"• AI Score: {result['summary']['avg_score']:.1f}/100")
    
    print(f"\n🎯 Top 10 Signals:")
    for i, sig in enumerate(result['signals'][:10], 1):
        score = sig['ai_score']
        indicator = "🔥" if score >= 80 else "⚡" if score >= 60 else "📊"
        print(f"\n{i}. {indicator} [{score}] {sig['signal']}")
        print(f"   {sig['desc']}")
        if sig.get('ai_reasoning'):
            print(f"   💡 {sig['ai_reasoning']}")

print("✅ Save & export functions loaded")


# ============================================================================
# CELL 7: Comparison & Screening Functions
# ============================================================================

def compare_stocks(symbols, period='1mo'):
    """Compare multiple stocks side-by-side"""
    results = []
    
    print(f"\n🔍 Comparing {len(symbols)} stocks...")
    
    for symbol in symbols:
        try:
            result = analyze_stock(symbol, period)
            results.append({
                'symbol': symbol,
                'price': result['price'],
                'change': result['change'],
                'total_signals': result['summary']['total_signals'],
                'bullish': result['summary']['bullish'],
                'bearish': result['summary']['bearish'],
                'avg_score': result['summary']['avg_score'],
                'rsi': result['indicators']['rsi'],
                'macd': result['indicators']['macd'],
                'adx': result['indicators']['adx']
            })
        except Exception as e:
            print(f"⚠️  Error with {symbol}: {e}")
    
    comparison_df = pd.DataFrame(results)
    comparison_df = comparison_df.sort_values('avg_score', ascending=False)
    
    print(f"\n{'='*80}")
    print("📊 COMPARISON RESULTS")
    print('='*80)
    print(comparison_df.to_string(index=False))
    
    filepath = f"{DATA_DIR}/exports/comparison_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
    comparison_df.to_csv(filepath, index=False)
    print(f"\n💾 Saved to {filepath}")
    
    return comparison_df


def screen_stocks(symbols, rsi_min=0, rsi_max=100, min_bullish_signals=0, min_score=0):
    """Screen stocks by technical criteria"""
    matches = []
    
    print(f"\n🔍 Screening {len(symbols)} stocks...")
    print(f"Criteria: RSI {rsi_min}-{rsi_max}, Min Bullish: {min_bullish_signals}, Min Score: {min_score}")
    
    for symbol in symbols:
        try:
            result = analyze_stock(symbol, period='1mo')
            
            if (rsi_min <= result['indicators']['rsi'] <= rsi_max and
                result['summary']['bullish'] >= min_bullish_signals and
                result['summary']['avg_score'] >= min_score):
                
                matches.append({
                    'symbol': symbol,
                    'price': result['price'],
                    'change': result['change'],
                    'rsi': result['indicators']['rsi'],
                    'bullish_signals': result['summary']['bullish'],
                    'avg_score': result['summary']['avg_score'],
                    'top_signal': result['signals'][0]['signal'] if result['signals'] else 'None'
                })
        except Exception as e:
            print(f"⚠️  {symbol}: {e}")
    
    matches_df = pd.DataFrame(matches)
    matches_df = matches_df.sort_values('avg_score', ascending=False)
    
    print(f"\n{'='*60}")
    print(f"✅ Found {len(matches)} matches")
    print('='*60)
    if not matches_df.empty:
        print(matches_df.to_string(index=False))
    
    if not matches_df.empty:
        filepath = f"{DATA_DIR}/exports/screener_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
        matches_df.to_csv(filepath, index=False)
        print(f"\n💾 Saved to {filepath}")
    
    return matches_df


def batch_analyze(symbols, period='1mo', delay=2):
    """Batch analyze multiple stocks with rate limiting"""
    import time
    
    results = {}
    
    print(f"\n📊 Batch analyzing {len(symbols)} stocks...")
    
    for i, symbol in enumerate(symbols, 1):
        print(f"\n[{i}/{len(symbols)}] Processing {symbol}...")
        try:
            result = analyze_and_save(symbol, period)
            results[symbol] = result
            
            if i < len(symbols):
                time.sleep(delay)
        except Exception as e:
            print(f"❌ Error with {symbol}: {e}")
            results[symbol] = None
    
    print(f"\n{'='*60}")
    print(f"✅ Batch analysis complete: {len([r for r in results.values() if r])} successful")
    print('='*60)
    
    return results

print("✅ Comparison & screening functions loaded")


# ============================================================================
# CELL 8: Interactive Dashboard (Plotly)
# ============================================================================

def create_dashboard(result):
    """Create interactive Plotly dashboard"""
    try:
        import plotly.graph_objects as go
        from plotly.subplots import make_subplots
    except ImportError:
        print("⚠️  plotly not installed. Run: pip install plotly")
        return
    
    df = result['data']
    symbol = result['symbol']
    
    fig = make_subplots(
        rows=4, cols=1,
        shared_xaxes=True,
        vertical_spacing=0.05,
        subplot_titles=(f'{symbol} Price & Moving Averages', 'RSI', 'MACD', 'Volume'),
        row_heights=[0.4, 0.2, 0.2, 0.2]
    )
    
    # Price & MAs
    fig.add_trace(go.Candlestick(
        x=df.index,
        open=df['Open'],
        high=df['High'],
        low=df['Low'],
        close=df['Close'],
        name='Price'
    ), row=1, col=1)
    
    fig.add_trace(go.Scatter(x=df.index, y=df['SMA_20'], name='SMA 20', line=dict(color='orange', width=1)), row=1, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=df['SMA_50'], name='SMA 50', line=dict(color='blue', width=1)), row=1, col=1)
    
    if 'SMA_200' in df.columns and not df['SMA_200'].isna().all():
        fig.add_trace(go.Scatter(x=df.index, y=df['SMA_200'], name='SMA 200', line=dict(color='red', width=2)), row=1, col=1)
    
    # Bollinger Bands
    fig.add_trace(go.Scatter(x=df.index, y=df['BB_Upper'], name='BB Upper', 
                            line=dict(color='gray', width=1, dash='dash'), opacity=0.5), row=1, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=df['BB_Lower'], name='BB Lower', 
                            line=dict(color='gray', width=1, dash='dash'), opacity=0.5), row=1, col=1)
    
    # RSI
    fig.add_trace(go.Scatter(x=df.index, y=df['RSI'], name='RSI', line=dict(color='purple')), row=2, col=1)
    fig.add_hline(y=70, line_dash="dash", line_color="red", opacity=0.5, row=2, col=1)
    fig.add_hline(y=30, line_dash="dash", line_color="green", opacity=0.5, row=2, col=1)
    
    # MACD
    fig.add_trace(go.Scatter(x=df.index, y=df['MACD'], name='MACD', line=dict(color='blue')), row=3, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=df['MACD_Signal'], name='Signal', line=dict(color='orange')), row=3, col=1)
    fig.add_trace(go.Bar(x=df.index, y=df['MACD_Hist'], name='Histogram', marker_color='gray'), row=3, col=1)
    
    # Volume
    colors = ['red' if close < open_ else 'green' for close, open_ in zip(df['Close'], df['Open'])]
    fig.add_trace(go.Bar(x=df.index, y=df['Volume'], name='Volume', marker_color=colors), row=4, col=1)
    fig.add_trace(go.Scatter(x=df.index, y=df['Volume_MA_20'], name='Vol MA 20', 
                            line=dict(color='orange', width=1)), row=4, col=1)
    
    fig.update_layout(
        title=f"{symbol} Technical Analysis Dashboard",
        height=1000,
        showlegend=True,
        xaxis_rangeslider_visible=False
    )
    
    fig.update_yaxes(title_text="Price", row=1, col=1)
    fig.update_yaxes(title_text="RSI", row=2, col=1)
    fig.update_yaxes(title_text="MACD", row=3, col=1)
    fig.update_yaxes(title_text="Volume", row=4, col=1)
    
    fig.show()
    
    html_file = f"{DATA_DIR}/exports/{symbol}_dashboard_{datetime.now().strftime('%Y%m%d_%H%M%S')}.html"
    fig.write_html(html_file)
    print(f"\n💾 Dashboard saved to {html_file}")
    
    return fig


def create_signals_chart(result):
    """Create chart showing signal distribution"""
    try:
        import plotly.graph_objects as go
    except ImportError:
        print("⚠️  plotly not installed")
        return
    
    category_counts = {}
    for signal in result['signals']:
        cat = signal['category']
        category_counts[cat] = category_counts.get(cat, 0) + 1
    
    categories = list(category_counts.keys())
    counts = list(category_counts.values())
    
    fig = go.Figure(data=[
        go.Bar(x=categories, y=counts, marker_color='steelblue')
    ])
    
    fig.update_layout(
        title=f"{result['symbol']} - Signal Distribution by Category",
        xaxis_title="Category",
        yaxis_title="Count",
        height=500
    )
    
    fig.show()
    return fig

print("✅ Dashboard functions loaded")


# ============================================================================
# CELL 9: Historical Tracking & Analysis
# ============================================================================

def load_historical_analysis(symbol, date_str):
    """Load previously saved analysis"""
    pattern = f"{DATA_DIR}/signals/{date_str}/{symbol}-signals-*.json"
    import glob
    
    files = glob.glob(pattern)
    if not files:
        return None
    
    latest_file = sorted(files)[-1]
    
    with open(latest_file, 'r') as f:
        return json.load(f)


def compare_historical(symbol, days_back=7):
    """Compare current vs historical signals"""
    current = analyze_stock(symbol, period='1mo')
    
    historical = []
    for i in range(1, days_back + 1):
        date_str = (datetime.now() - timedelta(days=i)).strftime('%Y-%m-%d')
        hist_data = load_historical_analysis(symbol, date_str)
        if hist_data:
            historical.append({
                'date': date_str,
                'total_signals': hist_data['summary']['total_signals'],
                'bullish': hist_data['summary']['bullish'],
                'bearish': hist_data['summary']['bearish'],
                'avg_score': hist_data['summary']['avg_score']
            })
    
    if historical:
        hist_df = pd.DataFrame(historical)
        
        print(f"\n{'='*60}")
        print(f"📈 Historical Comparison: {symbol}")
        print('='*60)
        print(f"\nCurrent:")
        print(f"  Total Signals: {current['summary']['total_signals']}")
        print(f"  Bullish: {current['summary']['bullish']} | Bearish: {current['summary']['bearish']}")
        print(f"  Avg Score: {current['summary']['avg_score']:.1f}")
        
        print(f"\nPast {days_back} days average:")
        print(f"  Total Signals: {hist_df['total_signals'].mean():.1f}")
        print(f"  Bullish: {hist_df['bullish'].mean():.1f} | Bearish: {hist_df['bearish'].mean():.1f}")
        print(f"  Avg Score: {hist_df['avg_score'].mean():.1f}")
        
        change = current['summary']['avg_score'] - hist_df['avg_score'].mean()
        trend = "📈 IMPROVING" if change > 5 else "📉 DECLINING" if change < -5 else "➡️ STABLE"
        print(f"\nTrend: {trend} ({change:+.1f} points)")
        
        return {'current': current, 'historical': hist_df}
    else:
        print(f"\n⚠️  No historical data found for {symbol}")
        return {'current': current, 'historical': None}


def track_signal_accuracy(symbol, signal_name, days=30):
    """Track how often a specific signal appears"""
    occurrences = []
    
    for i in range(days):
        date_str = (datetime.now() - timedelta(days=i)).strftime('%Y-%m-%d')
        hist_data = load_historical_analysis(symbol, date_str)
        
        if hist_data:
            found = any(s['signal'] == signal_name for s in hist_data['signals'])
            occurrences.append({
                'date': date_str,
                'found': found,
                'price': hist_data['price']
            })
    
    if occurrences:
        df = pd.DataFrame(occurrences)
        frequency = (df['found'].sum() / len(df)) * 100
        
        print(f"\n📊 Signal Tracking: {signal_name}")
        print(f"Symbol: {symbol} | Period: {days} days")
        print(f"Frequency: {frequency:.1f}% ({df['found'].sum()}/{len(df)} days)")
        
        return df
    else:
        print(f"\n⚠️  No historical data found")
        return None

print("✅ Historical tracking functions loaded")


# ============================================================================
# CELL 10: Utilities & Quick Actions
# ============================================================================

def quick_analysis(symbol):
    """Quick analysis with minimal output"""
    result = analyze_stock(symbol, period='1mo')
    
    print(f"\n{result['symbol']}: ${result['price']:.2f} ({result['change']:+.2f}%)")
    print(f"Signals: {result['summary']['bullish']}↑ / {result['summary']['bearish']}↓")
    print(f"Score: {result['summary']['avg_score']:.0f}/100")
    print(f"Top: {result['signals'][0]['signal']}")
    
    return result


def get_top_signals(symbol, n=5):
    """Get just the top N signals"""
    result = analyze_stock(symbol, period='1mo')
    
    print(f"\n🎯 Top {n} Signals for {symbol}:")
    for i, sig in enumerate(result['signals'][:n], 1):
        print(f"{i}. [{sig['ai_score']}] {sig['signal']} - {sig['desc']}")
    
    return result['signals'][:n]


def watchlist_summary(symbols):
    """Quick summary of watchlist"""
    summary = []
    
    print(f"\n📊 Watchlist Summary ({len(symbols)} stocks)")
    print('='*60)
    
    for symbol in symbols:
        try:
            result = quick_analysis(symbol)
            summary.append(result)
        except Exception as e:
            print(f"{symbol}: ❌ Error - {e}")
    
    return summary


def find_signals_by_category(result, category):
    """Filter signals by category"""
    matching = [s for s in result['signals'] if s['category'] == category]
    
    print(f"\n🔍 {category} signals for {result['symbol']}:")
    for sig in matching:
        print(f"  • [{sig['ai_score']}] {sig['signal']}: {sig['desc']}")
    
    return matching


def export_all_formats(result):
    """Export to all available formats"""
    print(f"\n📦 Exporting {result['symbol']} to all formats...")
    
    files = save_results(result)
    excel_file = export_to_excel(result)
    csv_file = export_to_csv(result)
    
    print(f"\n✅ All exports complete:")
    print(f"  • JSON: {files['signals_json']}")
    print(f"  • CSV: {csv_file}")
    print(f"  • Excel: {excel_file}")
    print(f"  • Report: {files['report_txt']}")
    
    return {**files, 'excel': excel_file, 'csv': csv_file}


def cleanup_old_data(days_to_keep=30):
    """Clean up analysis files older than N days"""
    cutoff_date = datetime.now() - timedelta(days=days_to_keep)
    deleted_count = 0
    
    for subdir in ['signals', 'indicators']:
        data_path = Path(f"{DATA_DIR}/{subdir}")
        if data_path.exists():
            for date_folder in data_path.iterdir():
                if date_folder.is_dir():
                    try:
                        folder_date = datetime.strptime(date_folder.name, '%Y-%m-%d')
                        if folder_date < cutoff_date:
                            import shutil
                            shutil.rmtree(date_folder)
                            deleted_count += 1
                    except ValueError:
                        pass
    
    print(f"🗑️  Cleaned up {deleted_count} old date folders (kept last {days_to_keep} days)")
    return deleted_count

print("✅ Utility functions loaded")
print("\n🎉 All cells loaded! Ready to use:")
print("  • analyze_and_save('AAPL') - Complete analysis with auto-save")
print("  • compare_stocks(['AAPL', 'MSFT', 'GOOGL']) - Compare multiple")
print("  • screen_stocks([symbols], rsi_max=35) - Screen by criteria")
print("  • create_dashboard(result) - Interactive Plotly dashboard")
print("  • quick_analysis('TSLA') - Fast minimal output")

In [ ]:
# ============================================================================
# CELL 11: Advanced Screening & Custom Filters
# ============================================================================

def advanced_screener(symbols, criteria_func, description="Custom"):
    """Screen with custom function logic"""
    matches = []
    
    print(f"\n🔍 Advanced Screening: {description}")
    
    for symbol in symbols:
        try:
            result = analyze_stock(symbol, period='1mo')
            if criteria_func(result):
                matches.append({
                    'symbol': symbol,
                    'price': result['price'],
                    'score': result['summary']['avg_score'],
                    'rsi': result['indicators']['rsi'],
                    'top_signal': result['signals'][0]['signal']
                })
        except:
            pass
    
    df = pd.DataFrame(matches).sort_values('score', ascending=False)
    print(f"✅ Found {len(df)} matches\n")
    print(df.to_string(index=False) if not df.empty else "No matches")
    
    return df


# Predefined screener functions
def oversold_reversal(result):
    return (result['indicators']['rsi'] < 35 and 
            result['indicators']['macd'] > result['data'].iloc[-1]['MACD_Signal'] and
            result['summary']['bullish'] > result['summary']['bearish'])

def momentum_breakout(result):
    return (result['change'] > 2 and 
            result['indicators']['volume'] > result['data'].iloc[-1]['Volume_MA_20'] * 1.5 and
            result['indicators']['rsi'] > 50)

def strong_uptrend(result):
    return (result['data'].iloc[-1]['SMA_10'] > result['data'].iloc[-1]['SMA_20'] > 
            result['data'].iloc[-1]['SMA_50'] and
            result['indicators']['adx'] > 25)


def multi_criteria_screen(symbols, min_score=70, rsi_range=(30, 70), 
                         min_volume_ratio=1.5, trend='bullish'):
    """Screen with multiple technical criteria"""
    matches = []
    
    for symbol in symbols:
        try:
            result = analyze_stock(symbol, period='1mo')
            current = result['data'].iloc[-1]
            
            volume_ratio = result['indicators']['volume'] / current['Volume_MA_20']
            trend_check = (result['summary']['bullish'] > result['summary']['bearish'] 
                         if trend == 'bullish' else 
                         result['summary']['bearish'] > result['summary']['bullish'])
            
            if (result['summary']['avg_score'] >= min_score and
                rsi_range[0] <= result['indicators']['rsi'] <= rsi_range[1] and
                volume_ratio >= min_volume_ratio and
                trend_check):
                
                matches.append({
                    'symbol': symbol,
                    'score': result['summary']['avg_score'],
                    'rsi': result['indicators']['rsi'],
                    'vol_ratio': f"{volume_ratio:.1f}x"
                })
        except:
            pass
    
    return pd.DataFrame(matches).sort_values('score', ascending=False)

print("✅ Advanced screening functions loaded")


# ============================================================================
# CELL 12: Portfolio Analysis & Correlation
# ============================================================================

def analyze_portfolio(symbols, weights=None):
    """Analyze entire portfolio with position weighting"""
    if weights is None:
        weights = {s: 1/len(symbols) for s in symbols}
    
    portfolio_data = []
    total_score = 0
    
    print(f"\n📊 Portfolio Analysis ({len(symbols)} positions)")
    
    for symbol in symbols:
        try:
            result = analyze_stock(symbol, period='1mo')
            weight = weights.get(symbol, 0)
            weighted_score = result['summary']['avg_score'] * weight
            
            portfolio_data.append({
                'symbol': symbol,
                'weight': f"{weight*100:.1f}%",
                'price': result['price'],
                'change': result['change'],
                'score': result['summary']['avg_score'],
                'weighted_score': weighted_score,
                'bullish': result['summary']['bullish'],
                'bearish': result['summary']['bearish']
            })
            total_score += weighted_score
        except Exception as e:
            print(f"⚠️  {symbol}: {e}")
    
    df = pd.DataFrame(portfolio_data)
    
    print(f"\n{'='*70}")
    print(df.to_string(index=False))
    print(f"{'='*70}")
    print(f"Portfolio Score: {total_score:.1f}/100")
    
    return df


def correlation_analysis(symbols, period='3mo'):
    """Analyze price correlation between symbols"""
    price_data = {}
    
    for symbol in symbols:
        try:
            result = analyze_stock(symbol, period=period)
            price_data[symbol] = result['data']['Close']
        except:
            pass
    
    df = pd.DataFrame(price_data)
    correlation_matrix = df.corr()
    
    print(f"\n📈 Correlation Matrix:")
    print(correlation_matrix.round(2))
    
    return correlation_matrix


def find_diversification_candidates(portfolio_symbols, candidate_symbols):
    """Find stocks with low correlation to portfolio"""
    portfolio_prices = {}
    candidate_prices = {}
    
    for symbol in portfolio_symbols:
        result = analyze_stock(symbol, period='3mo')
        portfolio_prices[symbol] = result['data']['Close']
    
    portfolio_df = pd.DataFrame(portfolio_prices)
    portfolio_avg = portfolio_df.mean(axis=1)
    
    candidates = []
    for symbol in candidate_symbols:
        try:
            result = analyze_stock(symbol, period='3mo')
            correlation = portfolio_avg.corr(result['data']['Close'])
            
            if abs(correlation) < 0.5:
                candidates.append({
                    'symbol': symbol,
                    'correlation': f"{correlation:.2f}",
                    'score': result['summary']['avg_score']
                })
        except:
            pass
    
    return pd.DataFrame(candidates).sort_values('score', ascending=False)

print("✅ Portfolio analysis functions loaded")


# ============================================================================
# CELL 13: Machine Learning Signal Prediction (Optional)
# ============================================================================

def prepare_ml_features(result):
    """Extract features for ML model"""
    current = result['data'].iloc[-1]
    
    features = {
        'rsi': current['RSI'],
        'macd': current['MACD'],
        'macd_signal': current['MACD_Signal'],
        'adx': current['ADX'],
        'stoch_k': current['Stoch_K'],
        'bb_position': (current['Close'] - current['BB_Lower']) / (current['BB_Upper'] - current['BB_Lower']),
        'volume_ratio': current['Volume'] / current['Volume_MA_20'],
        'price_change': current['Price_Change'],
        'dist_sma_20': current.get('Dist_SMA_20', 0),
        'volatility': current['Volatility']
    }
    
    return features


def build_signal_predictor(historical_data):
    """Train simple ML model to predict signal success"""
    try:
        from sklearn.ensemble import RandomForestClassifier
        from sklearn.model_selection import train_test_split
    except ImportError:
        print("⚠️  scikit-learn not installed. Run: pip install scikit-learn")
        return None
    
    # Prepare data
    X = []
    y = []
    
    for data_point in historical_data:
        features = list(data_point['features'].values())
        outcome = data_point['outcome']  # 1 if profitable, 0 if not
        X.append(features)
        y.append(outcome)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    
    # Train model
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)
    
    accuracy = model.score(X_test, y_test)
    print(f"✅ Model trained. Accuracy: {accuracy:.1%}")
    
    return model


def predict_signal_success(result, model):
    """Predict if signals will be profitable"""
    if model is None:
        return None
    
    features = prepare_ml_features(result)
    feature_vector = [list(features.values())]
    
    probability = model.predict_proba(feature_vector)[0][1]
    prediction = model.predict(feature_vector)[0]
    
    return {
        'prediction': 'SUCCESS' if prediction == 1 else 'FAIL',
        'confidence': f"{probability:.1%}"
    }

print("✅ ML prediction functions loaded (optional)")


# ============================================================================
# CELL 14: Alerts & Notifications
# ============================================================================

def check_alert_conditions(result, conditions):
    """Check if any alert conditions are met"""
    alerts = []
    
    for condition in conditions:
        condition_type = condition['type']
        threshold = condition['threshold']
        
        if condition_type == 'rsi_oversold' and result['indicators']['rsi'] < threshold:
            alerts.append(f"RSI oversold: {result['indicators']['rsi']:.1f} < {threshold}")
        
        elif condition_type == 'rsi_overbought' and result['indicators']['rsi'] > threshold:
            alerts.append(f"RSI overbought: {result['indicators']['rsi']:.1f} > {threshold}")
        
        elif condition_type == 'high_score' and result['summary']['avg_score'] >= threshold:
            alerts.append(f"High score: {result['summary']['avg_score']:.1f} >= {threshold}")
        
        elif condition_type == 'price_change' and abs(result['change']) >= threshold:
            alerts.append(f"Large price move: {result['change']:+.1f}%")
    
    return alerts


def monitor_watchlist(symbols, alert_conditions):
    """Monitor watchlist and trigger alerts"""
    print(f"\n🔔 Monitoring {len(symbols)} symbols...")
    
    alerts_triggered = {}
    
    for symbol in symbols:
        try:
            result = analyze_stock(symbol, period='1mo')
            alerts = check_alert_conditions(result, alert_conditions)
            
            if alerts:
                alerts_triggered[symbol] = {
                    'price': result['price'],
                    'alerts': alerts
                }
                print(f"\n⚠️  {symbol} - {len(alerts)} alerts:")
                for alert in alerts:
                    print(f"   • {alert}")
        except:
            pass
    
    print(f"\n{'='*60}")
    print(f"Total alerts: {sum(len(v['alerts']) for v in alerts_triggered.values())}")
    
    return alerts_triggered


def create_alert_report(alerts_triggered):
    """Generate formatted alert report"""
    if not alerts_triggered:
        return "No alerts triggered"
    
    report = f"\n{'='*60}\nALERT REPORT - {datetime.now().strftime('%Y-%m-%d %H:%M')}\n{'='*60}\n\n"
    
    for symbol, data in alerts_triggered.items():
        report += f"{symbol} - ${data['price']:.2f}\n"
        for alert in data['alerts']:
            report += f"  • {alert}\n"
        report += "\n"
    
    return report

print("✅ Alert & notification functions loaded")


# ============================================================================
# CELL 15: Backtesting & Performance Tracking
# ============================================================================

def backtest_signal(symbol, signal_name, lookback_days=30, hold_days=5):
    """Simple backtest of signal performance"""
    results = []
    
    for i in range(lookback_days):
        date_str = (datetime.now() - timedelta(days=i)).strftime('%Y-%m-%d')
        hist_data = load_historical_analysis(symbol, date_str)
        
        if hist_data:
            signal_present = any(s['signal'] == signal_name for s in hist_data['signals'])
            entry_price = hist_data['price']
            
            # Check price after hold_days
            exit_date = (datetime.strptime(date_str, '%Y-%m-%d') + timedelta(days=hold_days)).strftime('%Y-%m-%d')
            exit_data = load_historical_analysis(symbol, exit_date)
            
            if exit_data and signal_present:
                exit_price = exit_data['price']
                return_pct = ((exit_price - entry_price) / entry_price) * 100
                
                results.append({
                    'entry_date': date_str,
                    'entry_price': entry_price,
                    'exit_price': exit_price,
                    'return': return_pct
                })
    
    if results:
        df = pd.DataFrame(results)
        win_rate = (df['return'] > 0).sum() / len(df) * 100
        avg_return = df['return'].mean()
        
        print(f"\n📊 Backtest: {signal_name} on {symbol}")
        print(f"Trades: {len(df)} | Win Rate: {win_rate:.1f}%")
        print(f"Avg Return: {avg_return:+.2f}%")
        
        return df
    
    return None


def track_prediction_accuracy(predictions_file='predictions.json'):
    """Track accuracy of past predictions"""
    if not os.path.exists(predictions_file):
        return None
    
    with open(predictions_file, 'r') as f:
        predictions = json.load(f)
    
    correct = sum(1 for p in predictions if p['actual'] == p['predicted'])
    total = len(predictions)
    accuracy = (correct / total * 100) if total > 0 else 0
    
    print(f"\n📊 Prediction Accuracy")
    print(f"Total: {total} | Correct: {correct} | Accuracy: {accuracy:.1f}%")
    
    return {'total': total, 'correct': correct, 'accuracy': accuracy}

print("✅ Backtesting functions loaded")


# ============================================================================
# CELL 16: Automation & Scheduling
# ============================================================================

def schedule_daily_analysis(symbols, run_time="16:30"):
    """Schedule daily analysis (requires apscheduler)"""
    try:
        from apscheduler.schedulers.background import BackgroundScheduler
    except ImportError:
        print("⚠️  apscheduler not installed. Run: pip install apscheduler")
        return None
    
    scheduler = BackgroundScheduler()
    
    def daily_job():
        print(f"\n{'='*60}")
        print(f"🕐 Running scheduled analysis: {datetime.now()}")
        print('='*60)
        
        for symbol in symbols:
            try:
                analyze_and_save(symbol, period='1mo')
            except Exception as e:
                print(f"❌ {symbol}: {e}")
    
    hour, minute = run_time.split(':')
    scheduler.add_job(daily_job, 'cron', hour=int(hour), minute=int(minute))
    scheduler.start()
    
    print(f"⏰ Scheduled daily analysis at {run_time} for {len(symbols)} symbols")
    return scheduler


def export_to_google_sheets(result, sheet_name='Technical Analysis'):
    """Export to Google Sheets (requires gspread & credentials)"""
    try:
        import gspread
        from oauth2client.service_account import ServiceAccountCredentials
    except ImportError:
        print("⚠️  gspread not installed. Run: pip install gspread oauth2client")
        return None
    
    try:
        scope = ['https://spreadsheets.google.com/feeds']
        creds = ServiceAccountCredentials.from_json_keyfile_name('credentials.json', scope)
        client = gspread.authorize(creds)
        
        sheet = client.open(sheet_name).sheet1
        sheet.append_row([
            datetime.now().strftime('%Y-%m-%d'),
            result['symbol'],
            result['price'],
            result['change'],
            result['indicators']['rsi'],
            result['signals'][0]['signal'],
            result['signals'][0]['ai_score']
        ])
        
        print(f"✅ Exported to Google Sheets: {sheet_name}")
        return True
    except Exception as e:
        print(f"⚠️  Google Sheets export failed: {e}")
        return False


def create_summary_email(results):
    """Create email summary of analysis results"""
    subject = f"Technical Analysis Summary - {datetime.now().strftime('%Y-%m-%d')}"
    
    body = f"""
Technical Analysis Summary
Date: {datetime.now().strftime('%Y-%m-%d %H:%M')}

Analyzed {len(results)} symbols:

"""
    
    for result in results[:10]:
        body += f"""
{result['symbol']}: ${result['price']:.2f} ({result['change']:+.2f}%)
  Score: {result['summary']['avg_score']:.0f}/100
  Top Signal: {result['signals'][0]['signal']}
  Signals: {result['summary']['bullish']}↑ / {result['summary']['bearish']}↓

"""
    
    return {'subject': subject, 'body': body}


def automated_workflow(symbols, export_formats=['json', 'excel'], 
                      enable_alerts=True, alert_conditions=None):
    """Complete automated workflow"""
    print(f"\n🤖 Running automated workflow for {len(symbols)} symbols...")
    
    results = []
    alerts = {}
    
    for symbol in symbols:
        try:
            result = analyze_and_save(symbol, period='1mo')
            results.append(result)
            
            if 'excel' in export_formats:
                export_to_excel(result)
            
            if enable_alerts and alert_conditions:
                symbol_alerts = check_alert_conditions(result, alert_conditions)
                if symbol_alerts:
                    alerts[symbol] = symbol_alerts
        except Exception as e:
            print(f"❌ {symbol}: {e}")
    
    # Summary
    print(f"\n{'='*60}")
    print(f"✅ Workflow complete: {len(results)} successful")
    if alerts:
        print(f"⚠️  Alerts triggered: {len(alerts)} symbols")
        for symbol, alert_list in alerts.items():
            print(f"  • {symbol}: {len(alert_list)} alerts")
    
    return {'results': results, 'alerts': alerts}

print("✅ Automation functions loaded")

print("\n" + "="*70)
print("🎉 ALL CELLS LOADED - COMPLETE TECHNICAL ANALYSIS SYSTEM")
print("="*70)
print("\nQuick Start:")
print("  result = analyze_and_save('AAPL')           # Analyze single stock")
print("  compare_stocks(['AAPL', 'MSFT', 'GOOGL'])   # Compare multiple")
print("  create_dashboard(result)                     # Interactive dashboard")
print("  watchlist_summary(['AAPL', 'MSFT', 'TSLA']) # Quick overview")
print("\nAdvanced:")
print("  advanced_screener(symbols, oversold_reversal)")
print("  analyze_portfolio(['AAPL', 'MSFT'], weights={'AAPL': 0.6, 'MSFT': 0.4})")
print("  monitor_watchlist(symbols, alert_conditions)")
print("  automated_workflow(symbols, export_formats=['json', 'excel'])")